In [1]:
import numpy as np
import dolfinx
from mpi4py import MPI
import ufl
import sympy 

In [2]:
domain = dolfinx.mesh.create_rectangle(MPI.COMM_WORLD,[[-1,-1],[1,1]],n=[10,10])

In [3]:
N = 2
M = 4
V = dolfinx.fem.TensorFunctionSpace(domain,('CG',1),shape=(N,M))
V_interp= dolfinx.fem.FunctionSpace(domain,('CG',1))

In [4]:
F = dolfinx.fem.Function(V_interp)
F.function_space

FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), FiniteElement('Lagrange', triangle, 1))

In [5]:
def l_f(x):
    #print(x)
    #print(x.shape)
    return x[0]
F.interpolate(l_f)
F


Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), FiniteElement('Lagrange', triangle, 1)), 0)

In [6]:
8*600

4800

In [7]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

X = ufl.SpatialCoordinate(V)

In [8]:
from ufl import Dx, dX


vars = {'m':None,'g':None}
ufl_constants = {k:dolfinx.fem.Constant(V,c=np.complex128(1) if v is None else v) for k,v in vars.items()}


In [9]:
dolfinx.fem.Constant(V,np.complex128(1))

Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 2)

In [10]:

i,j,k = ufl.indices(3)
H = -0.5/ufl_constants['m']*ufl.grad(u)[i,j,k]*ufl.grad(ufl.conj(v))[i,j,k]*ufl.dx+ 0.5*ufl_constants['g']*ufl.conj(v)[i,j]*u[i,j]*F*ufl.dx

In [11]:
ufl_constants 
H

Form([Integral(IndexSum(IndexSum(IndexSum(Product(Indexed(Grad(Conj(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 0, None))), MultiIndex((Index(8), Index(9), Index(10)))), Product(Indexed(Grad(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None)), MultiIndex((Index(8), Index(9), Index(10)))), Division(FloatValue(-0.5), Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 0)))), MultiIndex((Index(8),))), MultiIndex((Index(9),))), MultiIndex((Index(10),))), 'cell', Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), 'everywhere', {}, None), Integral(Product(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), FiniteElement('Lagrange', t

In [12]:
L = dolfinx.fem.form(H)

In [13]:
L

In [14]:
A=dolfinx.fem.petsc.assemble_matrix(L)

In [15]:
A.assemble()
import scipy
from scipy.sparse import csr_matrix
a = csr_matrix(A.getValuesCSR()[::-1])

In [16]:
a.todense()

matrix([[-0.99586667+0.j,  0.        +0.j,  0.        +0.j, ...,
          0.        +0.j,  0.        +0.j,  0.        +0.j],
        [ 0.        +0.j, -0.99586667+0.j,  0.        +0.j, ...,
          0.        +0.j,  0.        +0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j, -0.99586667+0.j, ...,
          0.        +0.j,  0.        +0.j,  0.        +0.j],
        ...,
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
         -0.5016    +0.j,  0.        +0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
          0.        +0.j, -0.5016    +0.j,  0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
          0.        +0.j,  0.        +0.j, -0.5016    +0.j]])

In [17]:
M=ufl_constants['m']

In [18]:
import sympy
g = sympy.symbols('g')
c = np.complex128

In [19]:
n = sympy.symbols('n')
syms= {n:1}

In [20]:
(1,2)+(2,3)

(1, 2, 2, 3)

In [21]:

A = [[c(1),c(2),c(3)],[0,1,2],[0,3,3.0*g]]
A=sympy.Array(A)


In [22]:
def func1(x):
    print(1,x)
    return(1-x)

sympy.lambdify(g,A.tolist())(func1(1))

1 1


[[1.0, 2.0, 3.0], [0, 1, 2], [0, 3, 0.0]]

In [23]:
ufl.grad(u),

(Grad(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None)),)

In [24]:
F=v*u
type(F)

ufl.tensors.ComponentTensor

In [25]:
print(isinstance(F,ufl.algebra.Product))
F.ufl_operands


False


(IndexSum(Product(Indexed(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 0, None), MultiIndex((Index(11), Index(13)))), Indexed(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None), MultiIndex((Index(13), Index(12))))), MultiIndex((Index(13),))),
 MultiIndex((Index(11), Index(12))))

In [26]:
type(ufl.grad(u))

ufl.differentiation.Grad

In [27]:
def k_func(uv:ufl.algebra.Product,i):
    #argument is either u*ufl.conj(v) or terms like this with derivatives
    u_func,v_func = uv.ufl_operands
    if not isinstance(u_func,ufl.differentiation.Grad):
        return -1j*ufl.grad(u_func)*v
    else:
        return u_func* 1j*ufl.grad(v)
    

In [28]:
kx,ky,kz = sympy.symbols('k_x,k_y,k_z')

m,g = sympy.symbols('m,g')

F = sympy.Function('f')
A= sympy.Array([[1*kx*kx+ky*kz+ky*m,0,0],[0,2*ky*g,0],[1,0,3*kz]])
ten_shape = A.shape
spatial_dim = 3
A

[[k_x**2 + k_y*k_z + k_y*m, 0, 0], [0, 2*g*k_y, 0], [1, 0, 3*k_z]]

In [29]:
disassemble_dict = {}
from sympy import degree_list,degree
import numpy as np
for i,val in enumerate(np.array(A).ravel()):
    # split up value into terms 
    loop_terms = (val,) if not isinstance(val,sympy.core.add.Add) else val.args
    for term in loop_terms:
        try:
            orders = degree_list(term,(kx,ky,kz))
        except TypeError as e:
            continue 
        print(orders)
        arr = disassemble_dict.get(sum(orders),np.zeros(ten_shape+(3,)*(sum(orders)),'O'))
        # k-s commute, and by convention we will always order them as k_x k_x ... k_x k_y k_y ... k_y k_z k_z ... k_z
        arr_i = np.unravel_index(i,ten_shape)+(0,)*orders[0]+(1,)*orders[1]+(2,)*orders[2]
        arr[arr_i] += sympy.lambdify((kx,ky,kz),term)(1,1,1)
        disassemble_dict[sum(orders)] = arr

(2, 0, 0)
(0, 1, 1)
(0, 1, 0)
(0, 1, 0)
(0, 0, 0)
(0, 0, 1)


In [30]:
disassemble_dict[2][:,:,2,2]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=object)

In [31]:
from numpy import isin


val = A[0,0]
type(val)
kx =sympy.symbols(r'k_x')

In [32]:
type(degree(A.tolist()[0][1],kx))

sympy.core.numbers.NegativeInfinity

In [33]:
sympy.symbols('n_{z-modes}')

n_{z-modes}

In [34]:
A/

[[k_x**2 + k_y*k_z + k_y*m, 0, 0], [0, 2*g*k_y, 0], [1, 0, 3*k_z]]

In [35]:
B = sympy.Array([[1*kx*kx+ky*kz+ky,0,0],[0,2*ky,0],[1,0,3*kz]])

In [36]:
A == B


False

In [37]:
D = {1:1,2:2,3:3}
DD = {3:3,2:2,1:1}

In [38]:
D == DD

True

In [39]:
ufl_constants

{'m': Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 0),
 'g': Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 1)}

In [40]:
A

[[k_x**2 + k_y*k_z + k_y*m, 0, 0], [0, 2*g*k_y, 0], [1, 0, 3*k_z]]

In [41]:
#A.subs({s: ufl_constants[s.name] for s in (m,g)})

In [42]:
R=sympy.lambdify((m,g,kx,ky,kz),A)(ufl_constants['m'],ufl_constants['g'],1,1,1)

In [43]:
ufl_constants['m']

Constant(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), (), 0)

In [100]:
R[0,0] 


A = np.arange(16).reshape(4,4).astype(np.complex128)

tA = ufl.as_tensor(A*x[0]*x[1]**2)
print(tA)

[
  [0, x[0] * x[1] ** 2, 2.0 * x[0] * x[1] ** 2, 3.0 * x[0] * x[1] ** 2],
  [4.0 * x[0] * x[1] ** 2, 5.0 * x[0] * x[1] ** 2, 6.0 * x[0] * x[1] ** 2, 7.0 * x[0] * x[1] ** 2],
  [8.0 * x[0] * x[1] ** 2, 9.0 * x[0] * x[1] ** 2, 10.0 * x[0] * x[1] ** 2, 11.0 * x[0] * x[1] ** 2],
  [12.0 * x[0] * x[1] ** 2, 13.0 * x[0] * x[1] ** 2, 14.0 * x[0] * x[1] ** 2, 15.0 * x[0] * x[1] ** 2]
]


In [96]:
np.array([a*x[0] for a in A.ravel()]).reshape(A.shape)

array([[Zero((), (), ()),
        Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),))),
        Product(FloatValue(2.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),)))),
        Product(FloatValue(3.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),))))],
       [Product(FloatValue(4.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),)))),
        Product(FloatValue(5.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),)))),
        Product(FloatValue(6.0), Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(0),)))),
        Product(Float

In [97]:
x = ufl.SpatialCoordinate(V)
print(tA*x[0])

{ A | A_{i_{52}, i_{53}} = x[0] * ([
  [0, 1.0, 2.0, 3.0],
  [4.0, 5.0, 6.0, 7.0],
  [8.0, 9.0, 10.0, 11.0],
  [12.0, 13.0, 14.0, 15.0]
])[i_{52}, i_{53}] }


In [86]:

u= ufl.TrialFunction(V)
v = ufl.TestFunction(V)
I = ufl.indices(2)
u[I].dx(1)*ufl.conj(v)[I]*ufl.dx

Form([Integral(IndexSum(IndexSum(Product(Indexed(Grad(Indexed(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None), MultiIndex((Index(44), Index(45))))), MultiIndex((FixedIndex(1),))), Indexed(Conj(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 0, None)), MultiIndex((Index(44), Index(45))))), MultiIndex((Index(44),))), MultiIndex((Index(45),))), 'cell', Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), 'everywhere', {}, None)])

In [83]:
u[0,0]

Indexed(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None), MultiIndex((FixedIndex(0), FixedIndex(0))))

In [73]:
V

FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={}))

In [62]:
u.dx(1)*ufl.conj(v)

ComponentTensor(IndexSum(Product(Indexed(ComponentTensor(Indexed(Grad(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 1, None)), MultiIndex((Index(21), Index(22), FixedIndex(1)))), MultiIndex((Index(21), Index(22)))), MultiIndex((Index(23), Index(25)))), Indexed(Conj(Argument(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(2, 4), symmetry={})), 0, None)), MultiIndex((Index(25), Index(24))))), MultiIndex((Index(25),))), MultiIndex((Index(23), Index(24))))

In [ ]:
def F(x):
    x = np.asarray(x)
    print(x.shape)
    if len(x.shape) == 1:
        x = x[np.newaxis,:]
        print(x.shape)
    x_cords = np.linspace(-10,10,100).astype(np.complex128)
    y_vals = x_cords*10
    return_arr = []
    for xx in x:
        return_arr.append(y_vals[np.argmin(np.abs(x_cords-xx[0]))]) # find closest point to interpolate
    return np.stack(return_arr)

In [ ]:
F([1,0,0])

(3,)
(1, 3)


array([9.09090909+0.j])

In [ ]:
X

SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 3))

In [ ]:
fem_F = dolfinx.fem.Function(V)

In [ ]:
fem_F.interpolate(simple_F)

RuntimeError: Interpolation data has the wrong shape/size.

In [ ]:
simple_F = lambda x:np.arange(6).reshape(2,3)

In [ ]:
fem_F

Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 3), TensorElement(FiniteElement('Lagrange', triangle, 1), shape=(3, 2), symmetry={})), 2)

In [ ]:
fem_F.interpolate(lambda x:0)

RuntimeError: bad_function_call

In [ ]:
F.x.array.shape

(968,)

In [ ]:
domain.geometry.x.shape

(121, 3)

In [ ]:
F.x.array

array([7.000e+00+0.j, 6.070e+02+0.j, 1.207e+03+0.j, 1.807e+03+0.j,
       2.407e+03+0.j, 3.007e+03+0.j, 3.607e+03+0.j, 4.207e+03+0.j,
       1.000e+00+0.j, 6.010e+02+0.j, 1.201e+03+0.j, 1.801e+03+0.j,
       2.401e+03+0.j, 3.001e+03+0.j, 3.601e+03+0.j, 4.201e+03+0.j,
       1.000e+01+0.j, 6.100e+02+0.j, 1.210e+03+0.j, 1.810e+03+0.j,
       2.410e+03+0.j, 3.010e+03+0.j, 3.610e+03+0.j, 4.210e+03+0.j,
       2.200e+01+0.j, 6.220e+02+0.j, 1.222e+03+0.j, 1.822e+03+0.j,
       2.422e+03+0.j, 3.022e+03+0.j, 3.622e+03+0.j, 4.222e+03+0.j,
       1.900e+01+0.j, 6.190e+02+0.j, 1.219e+03+0.j, 1.819e+03+0.j,
       2.419e+03+0.j, 3.019e+03+0.j, 3.619e+03+0.j, 4.219e+03+0.j,
       2.500e+01+0.j, 6.250e+02+0.j, 1.225e+03+0.j, 1.825e+03+0.j,
       2.425e+03+0.j, 3.025e+03+0.j, 3.625e+03+0.j, 4.225e+03+0.j,
       4.000e+01+0.j, 6.400e+02+0.j, 1.240e+03+0.j, 1.840e+03+0.j,
       2.440e+03+0.j, 3.040e+03+0.j, 3.640e+03+0.j, 4.240e+03+0.j,
       4.300e+01+0.j, 6.430e+02+0.j, 1.243e+03+0.j, 1.843e+03+

In [ ]:
np.max(F.x.array)

(4799+0j)

In [ ]:
ufl_constants['m']*0 == 0

True

In [ ]:
X[1]

Indexed(SpatialCoordinate(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 0)), MultiIndex((FixedIndex(1),)))

In [ ]:
ufl_constants['m'].value

array(1.+0.j)

In [ ]:
A = np.array([[1,2,3,ufl_constants['m']],[10,11,12,ufl_constants['g']]])

In [ ]:
ufl_A=ufl.as_tensor(A)

In [ ]:
ufl_A.ufl_constants

AttributeError: 'ListTensor' object has no attribute 'ufl_constants'

NotImplementedError: Cannot take length of non-vector expression.